### 1 - Import test data

In [207]:
import yfinance as yf
import pandas as pd

# dataF = yf.download("EURUSD=X", start="2023-9-15", end="2023-11-1", interval='15m')
# dataF.iloc[:,:]
#dataF.Open.iloc

### 2 - Define your signal function

In [208]:
def signal_generator(df):
    vwap = df['VWAP']
    rsi = df['RSI']
    close_price = df['Close']
    upper_bb = df['UpperBB']
    lower_bb = df['LowerBB']

    if close_price > vwap:
        vwap_signal = "Bullish"
    else:
        vwap_signal = "Bearish"

    if rsi > 70:
        rsi_signal = "Overbought"
    elif rsi < 30:
        rsi_signal = "Oversold"
    else:
        rsi_signal = "Neutral"

    if close_price > upper_bb:
        bb_signal = "Overbought"
    elif close_price < lower_bb:
        bb_signal = "Oversold"
    else:
        bb_signal = "Neutral"

    if vwap_signal == "Bullish" and rsi_signal == "Oversold" and bb_signal == "Oversold":
        signal = 2
    elif vwap_signal == "Bearish" and rsi_signal == "Overbought" and bb_signal == "Overbought":
        signal = 1
    else:
        signal = 0


    return signal



### 3 - Connect to the market and execute trades

In [209]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.endpoints.instruments import InstrumentsCandles
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
import talib
import pandas_ta as ta



In [210]:
access_token= "cd7e21b015cb734770f5fcfea9da9c9f-49a95de76ed5cab29ee0d33cee5f004d"
accountID = "101-001-27332384-001" #your account ID here
list_of_currency = [Pair.EUR_USD, Pair.EUR_GBP, Pair.USD_JPY, Pair.AUD_USD, Pair.USD_CAD, Pair.CAD_JPY, Pair.NZD_USD]
def get_candles(n, currency):
    client = API(access_token)
    request = InstrumentsCandles(instrument=currency, params={"granularity": Gran.M15, "count": n})
    response = client.request(request)
    candles = response['candles']
    return candles

candles = get_candles(3, Pair.EUR_USD)



In [211]:
def calculate_vwap(data):
    data['VWAP'] = (((data['High'] + data['Low'] + data['Close']) / 3) * data['Volume']).cumsum() / data['Volume'].cumsum()

    
# Calculate Bollinger Bands
def calculate_bollinger_bands(data, period=20, std_dev=2):
    data['SMA'] = sum(data) / len(data)
    data['UpperBB'] = data['SMA'] + (std_dev * data['Close'].rolling(window=period).std())
    data['LowerBB'] = data['SMA'] - (std_dev * data['Close'].rolling(window=period).std())

# Calculate RSI
def calculate_rsi(data, period=14):
    data['RSI'] = talib.RSI(data['Close'], timeperiod=period)

In [214]:
def trading_job(currency, string_currency):
    candles = get_candles(30, currency)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low', 'Volume'])
    i=0
    for candle in candles:
        dfstream.loc[i, 'Open'] = float(candle['mid']['o'])
        dfstream.loc[i, 'Close'] = float(candle['mid']['c'])
        dfstream.loc[i, 'High'] = float(candle['mid']['h'])
        dfstream.loc[i, 'Low'] = float(candle['mid']['l'])
        dfstream.loc[i, 'Volume'] = float(candle['volume'])
        i += 1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)
    dfstream['Volume'] = dfstream['Volume'].astype(float)
    calculate_vwap(dfstream)
    #calculate_bollinger_bands(dfstream)
    my_bbands = ta.bbands(dfstream.Close, length=14, std=2.0)
    dfstream=dfstream.join(my_bbands)

    calculate_rsi(dfstream)

    print(dfstream)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    # EXECUTING ORDERS
    print("signal: " + str(signal))
    client = API(access_token)

    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])

    SLBuy = float(str(candle['mid']['o']))-previous_candleR
    SLSell = float(str(candle['mid']['o']))+previous_candleR

    TPBuy = float(str(candle['mid']['o']))+previous_candleR*SLTPRatio
    TPSell = float(str(candle['mid']['o']))-previous_candleR*SLTPRatio

    #print(dfstream.iloc[:-1,:])
    #print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    #signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument=string_currency, units=-25000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument=string_currency, units=25000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

In [215]:
list_of_currency = [Pair.EUR_USD, Pair.EUR_GBP, Pair.USD_JPY, Pair.AUD_USD, Pair.USD_CAD, Pair.CAD_JPY, Pair.NZD_USD]
list_of_str_currency = ["EUR_USD", "EUR_GBP", "USD_JPY", "AUD_USD","USD_CAD","CAD_JPY","NZD_USD"]

for i in range (len(list_of_str_currency)):
    trading_job(list_of_currency[i],list_of_str_currency[i])


# scheduler = BlockingScheduler()
# for i in range (len(list_of_str_currency)):
#     scheduler.add_job(trading_job, 'cron', args = [list_of_currency[i],list_of_str_currency[i]], day_of_week='mon-fri', hour='00-23', minute='1,11,21,31,41,51', start_date='2022-01-12 12:00:00')
# scheduler.start()

       Open    Close     High      Low  Volume      VWAP  BBL_14_2.0  \
0   1.07070  1.07085  1.07098  1.07068   289.0  1.070837         NaN   
1   1.07086  1.07090  1.07095  1.07084   280.0  1.070866         NaN   
2   1.07090  1.07090  1.07096  1.07085   221.0  1.070877         NaN   
3   1.07087  1.07112  1.07114  1.07084   163.0  1.070903         NaN   
4   1.07114  1.07072  1.07129  1.07060   302.0  1.070895         NaN   
5   1.07071  1.07040  1.07074  1.07034   322.0  1.070813         NaN   
6   1.07038  1.07050  1.07054  1.07024   342.0  1.070744         NaN   
7   1.07051  1.07034  1.07051  1.07028   454.0  1.070674         NaN   
8   1.07034  1.07038  1.07054  1.07028   475.0  1.070628         NaN   
9   1.07037  1.07081  1.07094  1.07033   617.0  1.070640         NaN   
10  1.07080  1.07122  1.07126  1.07051   795.0  1.070706         NaN   
11  1.07125  1.07122  1.07147  1.07112   576.0  1.070774         NaN   
12  1.07122  1.07152  1.07159  1.07111   429.0  1.070825        

KeyError: 'UpperBB'